# 1.Import
Import fasta file with the sample proteins

In [3]:
import Proteosim_EA as ps
proteins = ps.read_fasta("data/sample_proteins.fasta")
len(proteins)

3

# 2.Protein digestion
use digest directory and Trypsin as enzyme

In [4]:
digestion_enzyme = ps.enzyme_cleavage_patterns['Trypsin']


In [ ]:
digested_peptide_map = ps.digest_protein_collection(proteins, digestion_enzyme)